<a href="https://colab.research.google.com/github/katrina906/CS6120-Summarization-Project/blob/main/abstractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.thepythoncode.com/article/text-summarization-using-huggingface-transformers-python
#https://huggingface.co/blog/how-to-generate

In [1]:
%%capture
!pip install transformers
!pip install import-ipynb
!pip install sentencepiece

In [1]:
from transformers import pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import torch
import import_ipynb
import numpy as np
import tensorflow as tf

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# TODO create configuration options - text cleaning, length (min? max? penalty?), search type
# TODO evaluate on all summaries in dataframe
# TODO create loop through configurations similar to extractive 
# TODO set up evaluations 

In [2]:
# load in functions from extract_summarization notebook
%cd "drive/MyDrive/Colab Notebooks"
from extractive_summarization import *
%cd ..

/content/drive/MyDrive/Colab Notebooks
importing Jupyter notebook from extractive_summarization.ipynb
/content/drive/MyDrive


In [79]:
df = data_setup(n = 10000) 
df = df.head(20)

In [69]:
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-small")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [80]:
df['encoded'] = df.sentences_cleaned.map(lambda row: tokenizer.encode("summarize: " + ' '.join(row), return_tensors="pt", max_length=1017, truncation=True))

In [ ]:
# instead of generating one model per observation (30 seconds to train one model; over 80 hours to train 10,000 separate models)
# calculate overall averages based on number of sentences etc. during eval, find outliers of summaries that are much longer or shorter than desired by true ratio 
  # disadvantages: less easy to customize parameters. max length (assuming most important info at beginning, could completely miss topics later in article. ok for bcg - pyramid principle)

In [82]:
%%time
df['outputs'] = df.encoded.map(lambda row: model.generate( 
    row, 
    max_length=150, # TODO pass different values depending on length of input (like extracted)
    min_length=40, # force model to have longer outputs - don't allow to generate EOS token before this length
    length_penalty=2.0)) 

CPU times: user 1min 22s, sys: 10.2 s, total: 1min 32s
Wall time: 1min 32s


In [81]:
%%time
df['outputs'] = df.encoded.map(lambda row: model.generate( 
    row, 
    max_length=150, # TODO pass different values depending on length of input (like extracted)
    min_length=40, # force model to have longer outputs - don't allow to generate EOS token before this length
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True))

CPU times: user 3min 18s, sys: 21.4 s, total: 3min 39s
Wall time: 3min 39s


In [75]:
(10000*90)/60/60 # 250 hours for 10,000  
# 1 min 30 seconds for 10 inputs
# more than douple for 20 inputs = 3 min 30 seconds
# greedy - half the time 

250.0

In [ ]:
# pyspark?? https://towardsdatascience.com/distributed-hugging-face-tokenizer-using-pyspark-4873c27cf018

In [76]:
df['predicted_summary'] = df.outputs.map(lambda row: tokenizer.decode(row[0]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#summarizer = pipeline("summarization", framework = 'pt')
#summarized = summarizer(article, min_length=0, max_length=100) # max length is number of words/tokens
# max number of tokens can consider: 1024. Need to limit to first 1024
  # ok because best info tends to be at front of articles (and bcg documents bc pyramid principle)
  # potential extension is to fine tune with a higher max allowed

In [ ]:
# https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/